# De novo design

### Developers: Mat Bieniek, Ben Cree, Rachael Pirie, Josh Horton

* Add R-groups in user-defined positions
* Output ADMET properties
* Perform constrained optimisation
* Score poses
* Send output to free energy calculations

Open source, customisable, fast/easy to use.

In [1]:
import copy

import prody
from rdkit import Chem
from rdkit.Chem import Draw

from rgroup import RMol, RList, build_molecules, fix_receptor, optimise_in_receptor, RGroups, sort_conformers

ModuleNotFoundError: No module named 'prody'

In [ ]:
# load the complex with the ligand
sys = prody.parsePDB('sarscov2/7l10.pdb')
#sys = prody.parsePDB('jnk1/2gmx.pdb')

rec = sys.select('not (nucleic or hetatm) and not water')
mol = sys.select('resname XEY')
#mol = sys.select('resname 877')

prody.writePDB('rec.pdb', rec)
#prody.writePDB('lig.pdb', mol)

In [ ]:
#!wget 'https://models.rcsb.org/v1/5nev/ligand?auth_seq_id=301&label_asym_id=E&encoding=sdf&filename=5nev_E_72L.sdf' -O lig.sdf
#!obabel jnk1/core.sdf -O jnk1/coreh.sdf -p 7
!obabel sarscov2/lig.pdb -O sarscov2/coreh.sdf -p 7
# ^manually modify the files if need be

In [ ]:
# load the ligand
# rdkit doesn't do a good job with protonation, 
#init_mol = Chem.MolFromPDBFile('lig.pdb', removeHs=False)
#init_mol = Chem.AddHs(init_mol, addCoords=True)

init_mol = Chem.SDMolSupplier('sarscov2/coreh.sdf', removeHs=False)[0]

# get the R group representation of the rdkit Mol
rinit_mol = RMol(init_mol)

In [ ]:
# select the atom to be replaced with the R group
rinit_mol.rep2D(idx=True, size=(500, 500))

In [ ]:
groups = RGroups.dataframe
R_group_propan = Chem.MolFromMolFile(groups.loc[groups['Name'] == 'Propan-1-ol']['Mol File'].values[0], removeHs=False)
R_group_methanol = Chem.MolFromMolFile(groups.loc[groups['Name'] == 'Methanol' ]['Mol File'].values[0], removeHs=False)
#R_group = Chem.MolFromMolFile('data/rgroups/molecules/aromatics/benzyl_alcohol.mol', removeHs=False)
#R_group = Chem.MolFromMolFile('jnk1/jnk1-1.mol', removeHs=False)

In [ ]:
# build a list of molecules
attachment_index = [40]
rmols = build_molecules(rinit_mol, attachment_index, [R_group_propan, R_group_methanol])

In [ ]:
rmols.rep2D()

In [ ]:
rmols[0].rep3D()

In [ ]:
rmols.toxicity()

In [ ]:
rmols.generate_conformers(num_conf=30, 
                         minimum_conf_rms=0.5, 
                        )
                        # flexible=[39, 18, 40, 16, 17, 19, 5, 6, 36, 2, 1, 33, 3, 4, 35, 34, 15, 38])

In [ ]:
#rmol.rep3D(template=True)

In [ ]:
rmols.GetNumConformers()

### Optimise conformers in context of protein

In [ ]:
# fix the receptor file 
fix_receptor("rec.pdb", "rec_h.pdb")
# load back into prody
rec_fix = prody.parsePDB("rec_h.pdb")

In [ ]:
# filter by the protein clashes
rmols[0].rep3D(prody=rec_fix)

In [ ]:
rmols.removeConfsClashingWithProdyProt(rec_fix)

In [ ]:
rmols[0].rep3D(prody=rec_fix)

In [ ]:
r = rmols[0]
r.RemoveAllConformers()

In [ ]:
# opt_mol, energies
energies = rmols.optimise_in_receptor(
    receptor_file="rec_h.pdb", 
    ligand_force_field="openff", 
    use_ani=False,
    sigma_scale_factor=0.8,
    relative_permittivity=4
)

In [ ]:
rmols[0].rep3D()

In [ ]:
rmols[0].to_file("optimised_coords.pdb")

In [ ]:
print(energies)

In [ ]:
final_energies = rmols.sort_conformers(energy_range=5)

In [ ]:
rmols[0].to_file("best_conformers0.pdb")
rmols[1].to_file("best_conformers1.pdb")

In [ ]:
print(final_energies)

In [ ]:
CNNscores = rmols.gnina(receptor_file="rec_h.pdb") 
CNNscores